Make sure to select the conda base kernel

In [2]:
import sys
print(sys.executable)
!conda list

/opt/conda/bin/python
# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
anyio                     3.5.0           py311h06a4308_0  
archspec                  0.2.1              pyhd3eb1b0_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0          py311h5eee18b_0  
arrow                     1.3.0                    pypi_0    pypi
asttokens                 2.0.5              pyhd3eb1b0_0  
async-lru                 2.0.4           py311h06a4308_0  
attrs                     23.1.0          py311h06a4308_0  
babel                     2.11.0          py311h06a4308_0  
beautifulsoup4            4.12.2          py311h06a4308_0  
bleach                    4.1.0              pyhd3eb1b0_0  
boltons                   23.0.0          py311h06a4308_

In [3]:
%conda init
%conda activate base
from skimage import io
import numpy as np
import os

no change     /opt/conda/condabin/conda
no change     /opt/conda/bin/conda
no change     /opt/conda/bin/conda-env
no change     /opt/conda/bin/activate
no change     /opt/conda/bin/deactivate
no change     /opt/conda/etc/profile.d/conda.sh
no change     /opt/conda/etc/fish/conf.d/conda.fish
no change     /opt/conda/shell/condabin/Conda.psm1
no change     /opt/conda/shell/condabin/conda-hook.ps1
no change     /opt/conda/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /opt/conda/etc/profile.d/conda.csh
no change     /home/codespace/.bashrc
No action taken.

Note: you may need to restart the kernel to use updated packages.

CondaError: Run 'conda init' before 'conda activate'


Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'skimage'

In [4]:
# filenames are structured with '_w1', '_w2', '_w3' to indicate channels.
directory = '../images/to_combine'
output_directory = '../images'
os.makedirs(output_directory)

In [28]:

# Create lists for different wavelength images
w1_images, w2_images, w3_images = [], [], []

# Populate the lists with corresponding image filenames
for filename in sorted(os.listdir(directory)):
    if ("_w1" in filename) and ('._helix' not in filename):
        w1_images.append(filename)
    elif ("_w2" in filename) and ('._helix' not in filename):
        w2_images.append(filename)
    elif ("_w3" in filename) and ('._helix' not in filename):
        w3_images.append(filename)

In [29]:
for w1, w2, w3 in zip(w1_images, w2_images, w3_images):
    # Read the images
    img_w1 = io.imread(os.path.join(directory, w1), as_gray=True)
    img_w2 = io.imread(os.path.join(directory, w2), as_gray=True)
    img_w3 = io.imread(os.path.join(directory, w3), as_gray=True)

    # Stack the images along the third axis to create an RGB image
    # Note: Ensure that the images are correctly scaled (0-1 range for floats, 0-255 for integers)
    img_w1 = img_w1 / img_w1.max()
    img_w2 = img_w2 / img_w2.max()
    img_w3 = img_w3 / img_w3.max()

    rgb_image = np.stack((img_w3, img_w2, img_w1), axis=-1)
    
    # Assume w1 is something like 'image1.tif'
    base_filename = os.path.splitext(w1)[0]  # This will remove the .tif and give 'image1'    
    output_filename = 'RGB_{}.tif'.format(base_filename)
    # Save or display the image
    io.imsave(os.path.join(output_directory, output_filename), (rgb_image * 255).astype(np.uint8))
    # io.imshow(rgb_image)  # Display the image